Github will render jupyter notebooks, but Bokeh plots won't work<br>
View this notebook with NBViewer:<br>
#view the slides: https://raw.githubusercontent.com/DouglasPatton/Hydro/master/pres.slides.html

# A Systematic, Flexible Approach to Connecting Rainfall and Land Cover to Flood Damages
## Douglas Patton
#### This model is a continuation of my work:
Patton, Douglas A, Rebecca Moore, Alan P Covich, and John C Bergstrom. 2013. “Ex-Post Reliability Assessment of Benefit Transfer Valuation Estimates of Wetland Ecosystem Service Supported by Okefenokee National Wildlife Refuge.” SSRN Working Paper. https://ssrn.com/abstract=2294080

## The System

1. Land cover influences the hydrologic cycle
2. The hydrologic cycle influences human decisions
3. Human decisions influence land cover...


Gober, P., and H. S. Wheater (2015),Debates—Perspectives on **socio-hydrology**: Modeling flood risk as a public policy problem,Water Resour.Res.,51, 4782–4788, doi:10.1002/2015WR016945.

## Modeling The System
### Land cover influences the hydrologic cycle
1. Rainfall runoff modeling
  
2. Connect spatial data to hydrologic model

3. Jointly estimate both models
 
   

### Rainfall Runoff Modeling
#### Overview of Relevant Hydrologic Models

1. Physical - mechanistic
2. Conceptual - simplified
3. Empirical - blackbox


Sitterson, J., Chris Knightes, R. Parmar, K. Wolfe, M. Muche, AND B. Avant. An Overview of Rainfall-Runoff Model Types. U.S. Environmental Protection Agency, Washington, DC, EPA/600/R-17/482, 2017.

### Rainfall Runoff Modeling
#### The Hydrologic Cycle<br>
![alt text](https://raw.githubusercontent.com/DouglasPatton/Hydro/master/img/HC.gif)
[Image from the University of Illinois WW2010 Project](http://ww2010.atmos.uiuc.edu/(Gh)/guides/mtr/hyd/smry.rxml)

### A Time Series Rainfall Runoff Model

1. Linear Rainfall-Runoff Model
    - 50+ lags of rainfall
    - other terms temp, AR
    
  <br> <br><br>
2. locally weighted lag model
  - KNN + WLS
  - hyperparameter tuning
      - endogenous time "window"
      - endogenous discharge "widow"

 

### Rainfall Runoff Modeling
#### Time Series Inspired Approach

### $f($precip$_{t-0}$,...,precip$_{t-L}$ ,runoff$_{t-v})$ = runoff$_{t}$
<br>
Rising limb: $L+1$ lagged precip terms<br>
Recession limb: 1 autoregressive$_v$ term

Base flow: include parameters shared across models

### Rainfall Runoff Modeling
Estimated lagged precipitation parameters by discharge level
![alt text](https://raw.githubusercontent.com/DouglasPatton/Hydro/master/img/okeRR_multiple_dailydata.png)

### Rainfall Runoff Modeling
runoff vs predicted runoff
![alt text](https://raw.githubusercontent.com/DouglasPatton/Hydro/master/img/RRresults.png)

### Rainfall Runoff Modeling
#### Current Modeling Goals
  - endogenize time "window"
  - endogenize storage "window"

### Rainfall Runoff Modeling
#### Locally Weighted Approach

  1. an observation, $i$, is centered, others indexed $j$
    1. compute $\Delta^t_{j}=$ *time*$_i$-*time*$_j$ 
    2. compute $\Delta^d_{j}=$*discharge*$_i$-*discharge*$_j$
    3. estimate WLS$_i$ where $w_j^{-1}=exp(Z^t*\Delta^t_{j}+Z^d*\Delta^d_{j})$
  
 


5. choose $Z^t,Z^d$ to minimize MSE summed across all $i$ models
    - where
        - *discharge* is estimated with a simple model
        - and $i$ can be all or sampled observations

### Rainfall Runoff Modeling
#### Locally Weighted approach

  - if $Z^t,Z^d$ are both 0
    - exp(0)=1
    - WLS = OLS for all $i$
    - maximum DF
    - all models identical


  - if $Z^d$ is big 
    - the model varies across discharge levels

  - if $Z^t$ is big
    - the model varies over time

  - if both $Z$ are big
    - fewer DF
    - more variation across models

### Rainfall Runoff Modeling
#### Locally Weighted approach
##### Training Z:
$T_{train}$ and $D_{train}$ are the number of time periods and discharge levels used to estimate Z.<br><br>
$I_{train}=T_{train}*D_{train}$ is the number of centered training models.

##### Predicting with Z: 
$T_{predict}$ and $D_{predict}$ are the number of time periods and discharge levels with a centered prediction model.<br><br>
$I_{predict}=T_{predict}*D_{predict}$ is the number of centered prediction models. 

### Rainfall Runoff Modeling
#### Locally Weighted approach

 -estimation strategies
   - include $l_1$ (lasso) regularization for lagged precip
  


   - include $l_2$ (ridge) regularization for $Z$


   - estimate $Z$ and regularization hyperparameters using cross validation or bootstrap


   - boost model by creating more training or prediction models for sites with high prediction error

## Modeling The System
### Connect spatial data to hydrologic model
**$g(landscape,precipitation)=f(rainfall)$**
- for each basin: 
     1. $f()$: collect precipitation parameters across time, discharge-level
       - an $L^+*T_{predict}*D_{predict}$ array
     2. $g()$: collect landscape parameters across time, dataset
       - land cover, elevation, fragmentation, soil/geology
     3. global linear algebra to predict $f()$ with $g()$ 

## Modeling The System
### Jointly estimate both models
##### $g(landscape,precipitation)=runoff$
  - recycle hyperparameters


  


## The System
### A systematic approach
#### developing flexible modules in python

In [1]:
site='02314500' #Fargo, Ga below the Okefenokee NWR
start='2011-01-01T00:00-0400'
start='2010-12-01T00:00-0400'
end='2011-03-15T00:00-0400'
paramlist=['00045', '00065'] #must be entered as strings

modelfeatures={'RRmodeltype':'distributed_lag', 'maxlag':90, 'startlag':0, 
               'incl_AR1':'yes','incl_constant':'yes','local':'no', 
               'local_count':0}#dictionary of model features

### A systematic approach
#### developing flexible modules in python

In [2]:
import USGShydro #https://nbviewer.jupyter.org/github/DouglasPatton/Hydro/blob/master/helpers.ipynbZ
sample=USGShydro.Hydrositedatamodel(site,start,end,paramlist,modelfeatures) 


Loading BokehJS ...

all series have matching times from start to end
all time steps are evenly spaced
The request has returned 9981 observations for 2 series


In [3]:
sample.simpleplot() 

In [4]:
sample.runTSmodel1()
sample.predictplot()

In [5]:
sample.runTSmodel1({'RRmodeltype':'distributed_lag', 'maxlag':90, 'startlag':0, 
               'incl_AR1':'no','incl_constant':'yes','local':'no', 
               'local_count':0})
sample.predictplot()